In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import imodelsx.process_results
import qa_questions
import random
import feature_spaces
import encoding_utils
import joblib
from qa_embedder import QuestionEmbedder
from tqdm import tqdm
import feature_spaces
import dvu
dvu.set_style()
fit_encoding = __import__('01_fit_encoding')

In [ ]:
class A:
    use_test_setup = False
    subject = 'UTS03'
    # feature_space = 'eng1000'
    feature_space = 'qa_embedder-10'
    # feature_space = 'qa_embedder-5'
    qa_questions_version = 'v1'
    num_ngrams_context = 10

    # feature_space = 'qa_embedder-25'
    # qa_questions_version = 'v1-ending'
    qa_embedding_model = 'meta-llama/Meta-Llama-3-8B-Instruct'
    # qa_embedding_model = 'mistralai/Mistral-7B-Instruct-v0.2'
    # qa_embedding_model = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
    trim = 5
    num_stories = -1
    seed_stories = 1


args = A()
story_names_train, story_names_test = fit_encoding.get_story_names(args)
story_name = story_names_test[:1]
s = story_name[0]

In [ ]:
allstories, vectors, wordseqs = feature_spaces.get_features(
    args.feature_space,
    allstories=story_name,
    qa_embedding_model=args.qa_embedding_model,
    qa_questions_version=args.qa_questions_version,
    downsample=False,
)
question_answers = vectors[s]
ds = wordseqs[s]

## Visualize downsampled feats

In [ ]:
features_downsampled = feature_spaces.downsample_word_vectors(
    allstories,
    vectors,
    wordseqs,
    strategy='exp',
)[s]


# # nromalize each row of features_downsampled
features_downsampled = features_downsampled / \
    np.linalg.norm(features_downsampled, axis=1)[:, np.newaxis]

# # normalize each column of vectors
vectors = vectors / np.linalg.norm(vectors, axis=0)[np.newaxis, :]

In [ ]:
for question_num in range(3):
    question_anwer_vals = question_answers[:, question_num]
    question_answers_downsampled = features_downsampled[:, question_num]

    plt.figure(figsize=(13, 4))

    # draw vertical line from 0 to each val
    trs = ds.tr_times

    # xlim = (0, int(max(wordseqs.data_times) - 1))
    xlim = (0, 50)
    for i in range(len(question_anwer_vals)):
        t = ds.data_times[i]
        if t >= xlim[0] and t <= xlim[1]:
            plt.plot([t, t],
                     [0, question_anwer_vals[i]], '-', alpha=0.5, color='gray')
            plt.plot(t, question_anwer_vals[i], 'o', color='gray')
    idxs = (trs >= xlim[0]) & (trs <= xlim[1])
    plt.plot(trs[idxs],
             question_answers_downsampled[idxs], '-o', color='C0')

    plt.xticks(trs)
    plt.xlim(xlim)

    plt.xlabel('Time (s)')
    plt.show()

In [ ]:
print(' '.join([' '.join(ds.chunks()[idx])
      for idx in np.arange(len(idxs))[idxs]]))

### Visualize feat examples

In [ ]:
ngrams_list = feature_spaces._get_ngrams_list_from_words_list(
    ds.data, ngram_size=args.num_ngrams_context)
questions = qa_questions.get_questions(args.qa_questions_version)

In [ ]:
qa = QuestionEmbedder(
    checkpoint=args.qa_embedding_model, questions=questions)

In [ ]:
# prompt = "<s>[INST]'Input text: watermelon\nQuestion: Does the input contain a measurement? Answer yes or no.[/INST]"
# qa.llm(prompt, use_cache=0)

In [ ]:
ngrams = np.array(ngrams_list)
ngrams = ['thirteen cigarretes', 'how could i make it home i was sobbing'] + \
    ngrams[np.arange(5, 30, 5)].tolist()

In [ ]:
embs = qa(ngrams)

In [ ]:
embs_df = pd.DataFrame(embs, columns=questions,
                       index=ngrams)
embs_df.style.background_gradient(cmap='viridis')